In [1]:
import cv2
import numpy as np
import time
import Vehicles
import pandas as pd 
from datetime import datetime
from time import time, sleep

In [13]:
vehicle_up = 0
vehicle_down = 0
vehicles = [] # list of objects 

max_vehicle_weight = 5 
vehicle_id = 1 


df1_column_names = ["Id", "Direction", "5 Minutes"]
df1 = pd.DataFrame(columns = df1_column_names)

df2_column_names = ["Flow (Veh/5 Minutes)", "5 Minutes"]
df1 = pd.DataFrame(columns = df2_column_names)

cap = cv2.VideoCapture('./Video_Data_Samples/sample3.mp4')
w = cap.get(3)
h = cap.get(4)
#print(w) 320 
#print(h) 240
frameArea = h*w
detection_area= frameArea/320

# up-down lines positions on the frame 
# the vehicles in this video are passing horizontally that is why we define the counting lines 
# horizontally 
detector_up = np.int(2*(h/5)) 
detector_down = np.int(3*(h/5))

# define tracking area 
up_limit = np.int(1*(h/5)) # fifth of the horizon would be enough 
down_limit = np.int(4*(h/5))

# determine the area between the four lines 

area1 = np.array([[0,detector_down],[w,detector_down]], np.int32).reshape((-1,1,2))
area2 = np.array([[0,detector_up],[w,detector_up]], np.int32).reshape((-1,1,2))
area3 = np.array([[0,up_limit],[w,up_limit]], np.int32).reshape((-1,1,2))
area4 = np.array([[0,down_limit],[w,down_limit]], np.int32).reshape((-1,1,2))

background_substractor = cv2.createBackgroundSubtractorMOG2()
# vehicles_count=0
while(cap.isOpened()):
    _, frame = cap.read()
    for i in vehicles:
        i.get_weight()  
    background_substractor_mask = background_substractor.apply(frame)
    try:
        cv2.imshow('First Frame', frame)
        cv2.imshow('Backgroud Subtraction', background_substractor_mask)
        # apply threshold on the mask
        _, imBin = cv2.threshold(background_substractor_mask, 200, 255, cv2.THRESH_BINARY)
        # remove noise 
        mask = cv2.morphologyEx(imBin, cv2.MORPH_OPEN,np.ones((3,3), np.uint8))
        mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, np.ones((11,11), np.uint8)) 
        cv2.imshow('Masked Image', cv2.resize(mask, (400, 400)))
    except:
        #If there is no more frames to show...
        print('EOF')
        print ('UP:', vehicle_up)
        print ('DOWN:', vehicle_down)
        print('Total: ', vehicle_up+ vehicle_down)
        break
        # look for contour
    contours, hierarchy = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    for cnt in contours:
         contour_area = cv2.contourArea(cnt)
         # determine the distance to start tracking 
         if contour_area > detection_area and contour_area < 20000:
             # determine the center of the contours
            M = cv2.moments(cnt)
             # get the cordinates of the center
            center_x = int(M['m10']/M['m00'])
            center_y = int(M['m01']/M['m00'])
            # calculate the up-right bounding rectangle
            x,y,w,h = cv2.boundingRect(cnt)
            # try to seperate two close objects 
            new = True
            for i in vehicles:
                if abs(x-i.getX()) <= w and abs(y-i.getY()) <= h:
                    # we are still inside the dimentions of the same contour
                    new = False
                    # Update the coordinates of the vehicle and reset weight 
                    i.updateCoords(center_x,center_y)
                   
                    if i.going_UP(detector_down, detector_up) == True:
                        vehicle_up += 1
                        #df1.append(i.getId(),"up",time.strftime("%c"))
                    elif i.going_DOWN(detector_down, detector_up) == True:
                        vehicle_down += 1      
                        #df1.append(i.getId(),"down",time.strftime("%c"))
                    df2=pd.DataFrame({'5 Minutes': [datetime.now().strftime('%d/%m/%Y %H:%M')],'Flow (Veh/5 Minutes)':[vehicle_up+vehicle_down]})
                    df1=df1.append(df2)
                    break  
                if i.getState() == '1':
                    if i.getDir() == 'down' and i.getY() > down_limit:
                        i.setDone()
                    elif i.getDir() == 'up' and i.getY() < up_limit:
                        i.setDone()
                if i.timedOut():
                    # Remove from the list
                    index = vehicles.index(i)
                    vehicles.pop(index)
                    del i
            if new == True:
                p = Vehicles.Vehicles(vehicle_id,center_x,center_y, max_vehicle_weight)
                vehicles.append(p)
                vehicle_id += 1   
            # draw rectangle    
            img = cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,0),2) 
    frame = cv2.polylines(frame, [area1], False, (255,0,0), thickness=2)
    frame = cv2.polylines(frame, [area2], False, (0,0,255), thickness=2)
    frame = cv2.polylines(frame, [area3], False, (255,255,255), thickness=1)
    frame = cv2.polylines(frame, [area4], False, (255,255,255), thickness=1)

    cv2.imshow('Frame', cv2.resize(frame, (400, 400)))    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break               
cap.release()
cv2.destroyAllWindows()

EOF
UP: 56
DOWN: 232
Total:  288


In [14]:
df1=df1.groupby(['5 Minutes']).max()

In [15]:
df2 = pd.DataFrame(columns = df2_column_names)
flag=False
val=0
for timeStamp, row in df1.iterrows():
    if(flag==False):
        val=row['Flow (Veh/5 Minutes)']
        flag=True;
        df2 = df2.append({'5 Minutes': timeStamp, 'Flow (Veh/5 Minutes)': val}, ignore_index=True)
    else:
        df2 = df2.append({'5 Minutes': timeStamp, 'Flow (Veh/5 Minutes)':row['Flow (Veh/5 Minutes)']-val }, ignore_index=True)
        val=row['Flow (Veh/5 Minutes)']

In [16]:
df2= df2.loc[:, ::-1] 

In [17]:
df2.to_csv('newDataset.csv', index=False)

In [18]:
df2

,5 Minutes,Flow (Veh/5 Minutes)
0,13/08/2022 14:43,18
1,13/08/2022 14:44,100
2,13/08/2022 14:45,93
3,13/08/2022 14:46,77


df2

In [19]:

# from datetime import datetime
# from time import time, sleep
# int cnt=0
# while True:
#     cnt=cnt+1
#     sleep(60 - time() % 60)
#     t=datetime.now()
#     df2=pd.DataFrame({'Timestamp': [t.strftime('%d/%m/%Y %I:%M')]})
#     df1.append(df2)
#     print(df1)
#     print(t.strftime('%d/%m/%Y %I:%M'))
#     if(cnt==2) break
    

# #   time.sleep(1)

In [20]:
# print(time()%60)

In [21]:
# df2_column_names = ["Flow", "Timestamp"]
# df1 = pd.DataFrame(columns = df2_column_names)

In [22]:
# t=datetime.now()
# df2=pd.DataFrame({'Timestamp': [datetime.now().strftime('%d/%m/%Y %I:%M')]})
# df1=df1.append(df2)

In [23]:
# while True:
#     sleep(0.5)
#     print(datetime.now().strftime('%S'), end=" ")
#     print(datetime.now().strftime('%f'))

In [24]:
# cnt=0
# while True:
#     cnt=cnt+1;
#     if((60 - time() % 60) == 0):
#         cnt=0
    
#     print(cnt)  

In [25]:
import glob

In [26]:
path = r'C:\Users\apeks\OneDrive\Desktop\akshat h2\roadTrafficForecast-master\Traffic_Data_cali' # use your path
all_files = glob.glob(path + "/*.csv")

li = []

for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)

frame = pd.concat(li, axis=0, ignore_index=True)

In [29]:
frame.to_csv('final.csv', index=False)